In [ ]:
# yeah this file is a lot cleaner than the first one
# more things work here
# and it's a lot better organized
import re
import simdjson as sj
import gzip
from tqdm import tqdm
import pandas as pd
from tqdm.notebook import tqdm
import swifter
import seaborn as sns
import numpy as np	
from nltk import sent_tokenize
import re
from sentence_transformers import SentenceTransformer
from wutils.general import save_pickle, load_pickle
from wutils.mat import MarkedMatrix
from scipy.spatial.distance import cdist
# for kernel
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from treeinterpreter import treeinterpreter as ti

In [ ]:
# changed from ./pickles/who_cord_df.pkl to ./pickles/newest_cord_df_ft.pkl
cord_df = load_pickle('./pickles/newest_cord_df_ft.pkl')

In [ ]:
# changed from ./pickles/newest_good_sample_df_ft.pkl to clean_good_df.pkl
good_df = load_pickle('./pickles/clean_good_df.pkl')

In [ ]:
# changed from ./pickles/newest_bad_df_ft.pkl to cleab_bad_df.pkl
bad_df = load_pickle('./pickles/clean_bad_df.pkl')

In [ ]:
# he's looking for a specific document here
# to show an example of clean text
good_df[good_df.cleanText.swifter.apply(lambda x: 'Virus-ridden particles are inhaled by others and come into contact with cells' in x)]

In [ ]:
# some function to parse urls
import urllib.parse

def get_hostname(url):
    o = urllib.parse.urlsplit(url)
    return o.hostname

In [ ]:
# guess he only wants news related to wuhan
# showing an example of how to filter text
def news_filter(text):
    keep_if = ['wuhan']
    text = text.lower()
    blacklist = ['u.s.', 'america', 'korea', 'china', 'mexico', 'australia', 'uk', 'u.k.', 'new york', 'los angeles', 'nfl', 'nba', 'mlb', 'epl']
    if any(x in text for x in keep_if):
        return True
    if any(x in text for x in blacklist):
        return False
    return True

In [ ]:
# these next few cells, I'm assuming he's just intializing the dataframes to be used
# I have no idea how some of this works
# and I don't really want to find out
# I am just going to trust that this works and that we will get functional data after all this is done
# good work Will!
good_df['hostname'] = good_df['url'].swifter.apply(get_hostname)
bad_df['hostname'] = bad_df['url'].swifter.apply(get_hostname)
good_df['mean_embed'] = good_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))
bad_df['mean_embed'] = bad_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))

is_AP = bad_df['cleanText'].swifter.apply(lambda x: '(AP)' in x)
print('Found: ', is_AP.sum())
bad_df = bad_df[~is_AP]

In [ ]:
KW = 'transmission'

In [ ]:
cord_df = cord_df.dropna()
cord_df['mean_embed'] = cord_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))
# this right here is simply filtering out to find the word vaccine
has_vax = cord_df[cord_df.title.swifter.apply(lambda x: KW in x.lower()) | cord_df.abstract.swifter.apply(lambda x: KW in x.lower())]
# vax_ref gets used quite a bit later, I assume he is first filtering out data from cord that contains
# information relevant to vaccines
# from the paper:
# "We extract a filtered set, which has articles with sports teams and popular cities/countries removed,
# and refer to it as the “Filtered News” dataset
# We also extract only articles that contain the word “vaccine” and call this the “Vaccine News” dataset"
vax_ref = np.vstack(has_vax['mean_embed'])

In [ ]:
# vax_good_df = good_df[good_df['cleanText'].swifter.apply(lambda x: KW in x.lower())]
# vax_bad_df = bad_df[bad_df['cleanText'].swifter.apply(lambda x: KW in x.lower())]
vax_good_df = good_df
vax_bad_df = bad_df
vax_good_df = vax_good_df[vax_good_df['cleanText'].swifter.apply(news_filter)]
vax_bad_df = vax_bad_df[vax_bad_df['cleanText'].swifter.apply(news_filter)]
vax_good_mat = np.vstack(vax_good_df.mean_embed)
vax_bad_mat = np.vstack(vax_bad_df.mean_embed)
vax_raw_mm = MarkedMatrix([('good', vax_good_mat), ('bad', vax_bad_mat)])

In [ ]:
# he's calculating distances between reference dataframe and to-be-labeled dataframe here I am assuming 
# below is a link to possible other metrics to use besides cosine
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html
# so, vgood2ref is the distance between vax_good_mat and vax_ref
vgood2ref = cdist(vax_good_mat, vax_ref, 'cosine')
print('done w/ good')
# vbad2ref is the distance between vax_bad_mat and vax_ref
vbad2ref = cdist(vax_bad_mat, vax_ref, 'cosine')
print('done w/ bad')
vax_mm = MarkedMatrix([('good', vgood2ref), ('bad', vbad2ref)])

In [ ]:
# none of the stuff written below here needs to be re-run when changing the distance function

In [ ]:
# Alt method
# has_vax = cord_df.sample(5000)
# vax_ref = np.vstack(has_vax['mean_embed'])

In [ ]:
# War on Terror Cells: Strategies to Eradicate "Novel Coronavirus" Effectively
np.where(~np.isfinite(vax_ref))

In [ ]:
# random forest: a collection of decision trees, where the final decision is the average of all the trees
# the idea is: the crowd is smarter
raw_tmodel = RandomForestClassifier(n_estimators=500, n_jobs=28)

In [ ]:
tmodel = RandomForestClassifier(n_estimators=500, n_jobs=28)

In [ ]:
# tmodel 2 is just a fatter randomForestClassified
# should work better supposedly?
tmodel2 = RandomForestClassifier(n_estimators=1500, n_jobs=28)

In [ ]:
tidx = 263
# he's finding the clean text for vax_good here
# not sure exactly what cleanText means but
vax_good_df.iloc[tidx].title, vax_good_df.iloc[tidx].cleanText

In [ ]:
# moved this function here from below
# commented out where it originally was
def tree_explain_good(idx):
    pred, bias, contrib = ti.predict(tmodel, vgood2ref[idx, :].reshape((1, -1)))
    print('CLEAN TEXT: ', vax_good_df.iloc[idx].cleanText)
    cmat = contrib.reshape((-1, 2))
    print('Prediction: ', pred)
    print('Bias: ', bias)
    import_bad = np.argsort(cmat[:, 0])[::-1]
    import_good = np.argsort(cmat[:, 1])[::-1]
    print('===== Towards Good: =====')
    for idx in import_good[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')
    print('===== Towards Bad: =====')
    for idx in import_bad[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')

def tree_explain_bad(idx):
    pred, bias, contrib = ti.predict(tmodel, vbad2ref[idx, :].reshape((1, -1)))
    print('CLEAN TEXT: ', vax_bad_df.iloc[idx].cleanText)
    cmat = contrib.reshape((-1, 2))
    print('Prediction: ', pred)
    print('Bias: ', bias)
    import_bad = np.argsort(cmat[:, 0])[::-1]
    import_good = np.argsort(cmat[:, 1])[::-1]
    print('===== Towards Bad: =====')
    for idx in import_bad[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')
    print('===== Towards Good: =====')
    for idx in import_good[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')

In [ ]:
# where the hell is this function defined??
# ok, I'm getting this weird error here:
# AttributeError: 'RandomForestClassifier' object has no attribute 'n_outputs_'
# maybe something needs to be run first before this is run?
# man, I fucked up though
# I ran this without noting the cell number, so I have no idea when this was run before
# I will come back to this after I run some other stuff and see if it works
# tree_explain_good(250)
# going to move this below

In [ ]:
# vax_mm.mat[~np.isfinite(vax_mm.mat)] = 0

In [ ]:
import time

In [ ]:
# I think here is where he starts evaluating how good the model functions
# I guess closer to 1 is good, and closer to 0 is bad?
# if I change the difference calculations above, maybe these numbers will change

# also: vax_mm was defined above as a marked matrix of vgood2ref as good, and vbad2ref as bad
# with vgood2ref as the distance between the good matrix and the vax ref
# and with vbad2ref as the distance between the bad matrix and the vax ref
# ideally, vgood2ref should have small distances
# while vbad2ref whould hav elarge distances
# I'm not sure how this works out number wise, but I'll have to think

In [ ]:
# ====== RUN FIRST TIME WITH COSINE ======
cosineResults = dict()

In [ ]:
print('trial_1:')
start = time.time()
res = vax_mm.single_split_classify(tmodel, return_labels=True)
print(res)
cosineResults['trial_1'] = res
end = time.time()
print('seconds: ', end - start)

In [ ]:
# running vax_mm with tmodel again
print('trial_2:')
start = time.time()
res = vax_mm.single_split_classify(tmodel, return_labels=True)
print(res)
cosineResults['trial_2'] = res
end = time.time()
print('seconds: ', end - start)

In [ ]:
# running vax_raw_mm with raw_t_model
print('trial_3:')
start = time.time()
res = vax_raw_mm.single_split_classify(raw_tmodel, return_labels=True)
print(res)
cosineResults['trial_3'] = res
end = time.time()
print('seconds: ', end - start)

In [ ]:
# running vax_mm with tmodel2
print('trial_4:')
start = time.time()
res = vax_mm.single_split_classify(tmodel2, return_labels=True)
print(res)
cosineResults['trial_4'] = res
end = time.time()
print('seconds: ', end - start)

In [ ]:
# running vax_raw_mm with tmodel2
print('trial_5:')
start = time.time()
res = vax_raw_mm.single_split_classify(tmodel2, return_labels=True)
print(res)
cosineResults['trial_5'] = res
end = time.time()
print('seconds: ', end - start)

In [ ]:
# print the contents of cosineResults
for e in cosineResults:
    print('===================')
    print(e)
    print(cosineResults[e])

In [ ]:
# ok here, we want to change from a simple distance calculation into a kernel

In [ ]:
vgood2ref = pairwise_kernels(vax_good_mat, vax_ref, metric='polynomial')
print('done w/ good')
vbad2ref = pairwise_kernels(vax_bad_mat, vax_ref, metric='polynomial')
print('done w/ bad')
vax_mm = MarkedMatrix([('good', vgood2ref), ('bad', vbad2ref)])

In [ ]:
# old distance calculation for reference
# so as of right now, vgood2ref and vbad2ref are a set of distances of each article from the reference articles
# and this distance is use to classify
# if we convert to kernels, we would need to convert positions into an array of numbers somehow
# can kernels do that?
# stay tuned to find out!

# vgood2ref = cdist(vax_good_mat, vax_ref, 'cosine')
# print('done w/ good')
# # vbad2ref is the distance between vax_bad_mat and vax_ref
# vbad2ref = cdist(vax_bad_mat, vax_ref, 'cosine')
# print('done w/ bad')
# vax_mm = MarkedMatrix([('good', vgood2ref), ('bad', vbad2ref)])

In [ ]:
# ok so this works if we put it here I guess
tree_explain_good(250)

In [ ]:
# so it seems that tree_explain_good works after all the training is done
tree_explain_good(107)

In [ ]:
# tree_explain_good(43)
tree_explain_good(80)

In [ ]:
# tree_explain_good(43)
tree_explain_good(50)

In [ ]:
# this hasn't been run before
# changing t-model to have fewer estimators
tmodel2 = RandomForestClassifier(n_estimators=500, n_jobs=24)

In [ ]:
# neither has this
# nothing below this has been run before
# train and evaluate
# vax_raw_mm with tmodel2
vax_raw_mm.single_split_classify(tmodel2, return_labels=True)

In [ ]:
# ok so tree_explain is defined all the way down here
# going to move this higher
# def tree_explain_good(idx):
#     pred, bias, contrib = ti.predict(tmodel, vgood2ref[idx, :].reshape((1, -1)))
#     print('CLEAN TEXT: ', vax_good_df.iloc[idx].cleanText)
#     cmat = contrib.reshape((-1, 2))
#     print('Prediction: ', pred)
#     print('Bias: ', bias)
#     import_bad = np.argsort(cmat[:, 0])[::-1]
#     import_good = np.argsort(cmat[:, 1])[::-1]
#     print('===== Towards Good: =====')
#     for idx in import_good[:5]:
#         print('-'*45)
#         print(f'CONTRIB: {cmat[idx]}')
#         print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
#         print(f'TITLE: {has_vax.iloc[idx].title}')
#     print('===== Towards Bad: =====')
#     for idx in import_bad[:5]:
#         print('-'*45)
#         print(f'CONTRIB: {cmat[idx]}')
#         print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
#         print(f'TITLE: {has_vax.iloc[idx].title}')

# def tree_explain_bad(idx):
#     pred, bias, contrib = ti.predict(tmodel, vbad2ref[idx, :].reshape((1, -1)))
#     print('CLEAN TEXT: ', vax_bad_df.iloc[idx].cleanText)
#     cmat = contrib.reshape((-1, 2))
#     print('Prediction: ', pred)
#     print('Bias: ', bias)
#     import_bad = np.argsort(cmat[:, 0])[::-1]
#     import_good = np.argsort(cmat[:, 1])[::-1]
#     print('===== Towards Bad: =====')
#     for idx in import_bad[:5]:
#         print('-'*45)
#         print(f'CONTRIB: {cmat[idx]}')
#         print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
#         print(f'TITLE: {has_vax.iloc[idx].title}')
#     print('===== Towards Good: =====')
#     for idx in import_good[:5]:
#         print('-'*45)
#         print(f'CONTRIB: {cmat[idx]}')
#         print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
#         print(f'TITLE: {has_vax.iloc[idx].title}')

In [ ]:
pred, bias, contrib = ti.predict(tmodel, vbad2ref[290, :].reshape((1, -1)))
# pred, bias, contrib = ti.predict(tmodel, vbad2ref[290, :])

In [ ]:
cmat = contrib.reshape((-1, 2))
import_bad = np.argsort(cmat[:, 0])[::-1]
import_good = np.argsort(cmat[:, 1])[::-1]

In [ ]:
# claims_df = load_pickle('./pickles/newest_claims_df_labelled.pkl')
# claims_df = claims_df.dropna()

# true_claims_df = claims_df[claims_df['label'] == 'true']
# false_claims_df = claims_df[claims_df['label'] == 'false']
# misleading_claims_df = claims_df[claims_df['label'] == 'misleading']

# true_claims_mat = np.vstack(list(true_claims_df['bert_embedding']))
# false_claims_mat = np.vstack(list(false_claims_df['bert_embedding']))

# true_claims2ref = cdist(true_claims_mat, vax_ref, 'cosine')
# false_claims2ref = cdist(false_claims_mat, vax_ref, 'cosine')

# claims_mm = MarkedMatrix([('true', true_claims_mat), ('false', false_claims_mat)])
# claims_mm2covid = MarkedMatrix([('true', true_claims2ref), ('false', false_claims2ref)])
# claims_mm.single_split_classify(tmodel, return_labels=True)
# tmodel_claims = RandomForestClassifier(n_estimators=500, n_jobs=24)
# claims_mm2covid.single_split_classify(tmodel_claims, return_labels=True)